In [1]:
import re

import pandas as pd

In [2]:
AGS_RE = re.compile('^(\d{5}).*', re.M)
data = []
PATH = 'raw/Pflegestatistik_2013_Niedersachsen.xls'

In [3]:
def get_values(df, mapping, prefix='personal', indicator='key', skip_on=None):
    ags = None
    ags_val_cache = set()
    for i, row in df.iterrows():
        ind = str(row[indicator])
        if not pd.isnull(ind) and AGS_RE.match(ind) is not None:
            ags = ind.replace(' ', '')
        if ags is None:
            continue
        if pd.isnull(row['key']):
            continue
        if skip_on is not None and row['key'] in skip_on:
            ags = None
            continue
        for k, v in mapping.items():
            if k in row['key']:
                if isinstance(row['value'], str):
                    val = row['value'].replace('-', '0').strip()
                    if val == '.':
                        val = None
                    else:
                        val = float(val)
                else:
                    val = row['value']
                cache_key = '%s-%s' % (ags, v)
                if cache_key not in ags_val_cache:
                    yield ags, prefix, v, val
                    ags_val_cache.add(cache_key)


In [4]:
df = pd.read_excel(PATH, sheetname='Vergütung-Kreise')

df = df.rename(columns={
    'Niedersachsen             ': 'key',
    'Pflegestatistik über stationäre Einrichtungen': 'value'
})

df

cost_mapping = {
    'Pflegeklasse 1': 'costs_nursing_class_1',
    'Pflegeklasse 2': 'costs_nursing_class_2',
    'Pflegeklasse 3': 'costs_nursing_class_3',
    'Verpflegung': 'food',
}


data.extend(list(get_values(df, cost_mapping, prefix='costs', skip_on=['Vergütung für Kurzzeitpflege  '])))
df.head()

,key,value,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,03,S 6 Vergütungen am 15.12.2013 in Pflegeheimen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Blatt 1,S 6.1 Durchschnittliche Vergütungen nach dere...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Art und Trägergruppen der Einrichtung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,EUR pro Person und Tag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Durch-,Davon nach dem Angebot der Einrichtung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = pd.read_excel(PATH, sheetname='Plätze-Betten')

df = df.rename(columns={
    'Niedersachsen             ': 'key',
    'Pflegestatistik über stationäre Einrichtungen': 'value'
})

df

bed_mapping = {
  '1-Bett-Zimmern': '1-bed',
  '2-Bett-Zimmern': '2-bed',
  '3-Bett-Zimmern': '3-bed',
  '4 und mehr-Bett-Zimmern': '4-bed'
}


data.extend(list(get_values(df, bed_mapping, prefix='beds')))
df.head()

,key,value,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,03,S 2 Verfügbare Plätze am 15.12.2013 in Pflege...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Blatt 1,S 2.2 Verfügbare Plätze nach Art der Plätze u...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,sowie nach Trägergruppen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Davon nach dem Angebot der Einrichtung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Verfüg-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = pd.read_excel(PATH, sheetname='Personal-Kreise')

df = df.rename(columns={
    'Niedersachsen             ': 'key',
    'Unnamed: 2': 'value'
})


personal_mapping = {
    'Vollzeitbeschäftigt': 'fully_employed',
    'Teilzeitbeschäftigt': 'part-time',
    'Pflege und Betreuung': 'nursing_and_care',
    'soziale Betreuung': 'social_care',
    'zusätzliche Betreuung (§ 87b SGB XI)': 'other_care',
    'Personal insgesamt': 'total_personnel'
}

data.extend(list(get_values(df, personal_mapping, prefix='personal')))
df.head()

,key,Pflegestatistik über stationäre Einrichtungen,value,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,03,S 3 Personal am 15.12.2013 in Pflegeheimen,NaN,NaN,NaN,NaN,NaN,NaN
1,Blatt 1,"S 3.2 Personal nach Beschäftigungsverhältnis,...",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Arbeitsanteil für das Pflegeheim sowie nach Tr...,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Davon nach dem Arbeitsanteil für das Pflegehei...,NaN,NaN,NaN,NaN
4,Beschäftigungsverhältnis/,NaN,Personal,NaN,NaN,NaN,NaN,NaN


In [7]:
df = pd.read_excel(PATH, sheetname='Bedürftige-Kreise')

df = df.rename(columns={
    'Niedersachsen             ': 'key',
    'Pflegestatistik über stationäre Einrichtungen': 'value'
})


def get_values(df, mapping, prefix='personal', indicator='key', value_pattern=''):
    ags = None
    ags_val_cache = set()
    current_key = None
    for i, row in df.iterrows():
        ind = str(row[indicator])
        if not pd.isnull(ind) and AGS_RE.match(ind) is not None:
            ags = ind.replace(' ', '')
        if ags is None:
            continue
        if pd.isnull(row['key']):
            continue
        for k, v in mapping.items():
            if k in row['key']:
                current_key = v
                break
        if value_pattern in row['key'] and current_key is not None:
            if isinstance(row['value'], str):
                val = row['value'].replace('-', '0').strip()
                if val == '.':
                    val = None
                else:
                    val = float(val)
            else:
                val = row['value']
            cache_key = '%s-%s' % (ags, current_key)
            if cache_key not in ags_val_cache:
                yield ags, prefix, current_key, val
                ags_val_cache.add(cache_key)
            current_key = None


recipient_mapping = {
    'Pflegestufe I ': 'recipients_nursing_class_1',
    'Pflegestufe II ': 'recipients_nursing_class_2',
    'Pflegestufe III': 'recipients_nursing_class_3',
    'Pflegestufe zugeordnet': 'recipients_nursing_class_unknown',
}


data.extend(list(get_values(df, recipient_mapping, prefix='recipients', value_pattern='Zusammen ...')))
df.head()

,key,value,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,03,S 4 Pflegebedürftige am 15.12.2013 in Pflegeh...,NaN,NaN,NaN,NaN,NaN,NaN
1,Blatt 1,"S 4.2 Pflegebedürftige nach Pflegestufen, Alt...",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,nach Geschlecht und Trägergruppen der Einrichtung,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Davon nach Art der Pflegeleistung,NaN,NaN,NaN,NaN,NaN
4,Pflegestufen,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = pd.DataFrame(data)
df = df.rename(columns={
    0: 'ags',
    1: 'type',
    2: 'key',
    3: 'value'
})
df['state'] = 'Niedersachsen'
df.to_csv('csvs/niedersachsen.csv', index=False)
df.head()

,ags,type,key,value,state
0,03101,costs,costs_nursing_class_1,44.50,Niedersachsen
1,03101,costs,costs_nursing_class_2,57.65,Niedersachsen
2,03101,costs,costs_nursing_class_3,71.79,Niedersachsen
3,03101,costs,food,17.54,Niedersachsen
4,03102,costs,costs_nursing_class_1,42.71,Niedersachsen
